# Sudoku-11-Choco-Csharp : Solveur Choco via IKVM

**Navigation** : [<< OR-Tools](Sudoku-10-ORTools-Csharp.ipynb) | [Index](README.md) | [Z3 >>](Sudoku-12-Z3-Csharp.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Utiliser** Choco-solver depuis C# via le bridge IKVM
2. **Modeliser** un Sudoku comme un CSP avec Choco (variables, domaines, contraintes)
3. **Configurer** differentes strategies de recherche (FirstFail, DomOverWDeg)
4. **Comparer** Choco avec d'autres solveurs de contraintes

**Duree estimee** : ~30 min | **Prerequis** : [Sudoku-0 Environment](Sudoku-0-Environment-Csharp.ipynb)

---

Ce notebook implemente un solveur Sudoku utilisant **Choco-solver**, une librairie Java de Programmation par Contraintes, appelee depuis C# via **IKVM** (Java/.NET bridge).

## Introduction : Choco-solver

[Choco-solver](https://choco-solver.org/) est une librairie open-source Java de resolution de problemes de Programmation par Contraintes (CP), developpee par l'equipe TASC de l'Universite de Nantes.

### Pourquoi utiliser Choco depuis C# ?

- **Acces a un solveur CP mature** : Choco existe depuis 1999, tres documente
- **Contraintes globales optimisees** : `allDifferent`, `cumulative`, `circuit`, etc.
- **Strategies de recherche avancées** : DomOverWDeg, Impact-based search
- **Pedagogie** : Excellent pour comprendre la programmation par contraintes

## Configuration IKVM

IKVM est un bridge qui permet d'utiliser des librairies Java depuis .NET. Pour utiliser Choco depuis C#, nous avons besoin de :

1. **IKVM.OpenJDK.Core.dll** : Runtime Java pour .NET
2. **choco-solver.jar** : La librairie Choco compilee en DLL via IKVM

> **Note** : Ce notebook suppose que les assemblies IKVM et Choco sont disponibles dans le projet.

In [ ]:
#r "nuget: Sudoku.Shared, 1.0.0"

// Imports Choco via IKVM (espaces de noms Java mappes vers .NET)
using org.chocosolver.solver;
using org.chocosolver.solver.variables;
using org.chocosolver.solver.constraints;
using org.chocosolver.solver.search.strategy.selectors.variables;
using org.chocosolver.solver.search.strategy.selectors.values;
using Sudoku.Shared;
using System;
using System.Linq;
using System.Collections.Generic;

Console.WriteLine("Choco-solver via IKVM - Pret pour resolution Sudoku");

## 1. Implementation Simple : ChocoSimpleSolver

La premiere implementation utilise Choco de maniere directe sans optimisations particulieres.

In [ ]:
public class ChocoSimpleSolver : ISudokuSolver
{
    public SudokuGrid Solve(SudokuGrid s)
    {
        var model = new Model("Sudoku Solver - Simple");
        
        // Creer les 81 variables (1-9)
        var cellVariables = model.intVarMatrix("cells", 9, 9, 1, 9);
        
        var constraints = new List<Constraint>();
        
        // Contraintes pour les lignes et les colonnes
        for (int i = 0; i < 9; i++)
        {
            // Lignes : toutes les valeurs differentes
            constraints.Add(model.allDifferent(cellVariables[i]));
            // Colonnes : toutes les valeurs differentes
            constraints.Add(model.allDifferent(GetColumn(cellVariables, i)));
        }

        // Contraintes pour les blocs 3x3
        for (int blockRow = 0; blockRow < 3; blockRow++)
        {
            for (int blockCol = 0; blockCol < 3; blockCol++)
            {
                constraints.Add(model.allDifferent(GetBlock(cellVariables, blockRow, blockCol)));
            }
        }

        // Appliquer les valeurs initiales du Sudoku
        for (int row = 0; row < 9; row++)
        {
            for (int col = 0; col < 9; col++)
            {
                if (s.Cells[row, col] != 0)
                {
                    constraints.Add(model.arithm(cellVariables[row][col], "=", s.Cells[row, col]));
                }
            }
        }

        // Poster toutes les contraintes
        foreach (var constraint in constraints)
        {
            constraint.post();
        }

        // Resolution
        var solver = model.getSolver();
        if (solver.solve())
        {
            // Remplir la grille avec la solution
            for (int row = 0; row < 9; row++)
            {
                for (int col = 0; col < 9; col++)
                {
                    s.Cells[row, col] = cellVariables[row][col].getValue();
                }
            }
        }
        
        return s;
    }
    
    private IntVar[] GetColumn(IntVar[][] grid, int col)
    {
        return Enumerable.Range(0, 9).Select(row => grid[row][col]).ToArray();
    }

    private IntVar[] GetBlock(IntVar[][] grid, int blockRow, int blockCol)
    {
        return Enumerable.Range(0, 3)
            .SelectMany(i => Enumerable.Range(0, 3)
                .Select(j => grid[blockRow * 3 + i][blockCol * 3 + j]))
            .ToArray();
    }
}

Console.WriteLine("ChocoSimpleSolver defini.");

## 2. Implementation Optimisee : ChocoSolverVariableSelector

Cette version utilise des heuristiques de recherche avancees pour ameliorer les performances :

- **FirstFail** : Choisir la variable avec le plus petit domaine (MRV)
- **IntDomainMin** : Choisir la plus petite valeur disponible

In [ ]:
public class ChocoSolverVariableSelector : ISudokuSolver
{
    protected const int GridSize = 9;
    protected const int BlockSize = 3;
    protected IntVar[] FlatCells { get; set; } = Array.Empty<IntVar>();

    public virtual SudokuGrid Solve(SudokuGrid grid)
    {
        ValidateInput(grid);

        var model = new Model("Solveur Sudoku - Optimise");

        // Creer les variables cellulaires
        var cellVariables = CreateCellVariables(model, grid);
        FlatCells = Flatten(cellVariables);

        // Appliquer les contraintes
        ApplyConstraints(model, cellVariables);

        // Configurer le solveur avec la strategie de recherche
        var solver = GetSolver(model, cellVariables);

        // Resoudre
        if (solver.solve())
        {
            return ExtractSolution(grid, cellVariables);
        }
        else
        {
            throw new Exception("Aucune solution trouvee.");
        }
    }

    protected virtual void ValidateInput(SudokuGrid grid)
    {
        for (int i = 0; i < GridSize; i++)
        {
            if (HasDuplicatesInRow(grid.Cells, i) || HasDuplicatesInColumn(grid.Cells, i))
            {
                throw new ArgumentException($"Grille invalide : doublons ligne/colonne {i + 1}");
            }
        }
        for (int br = 0; br < BlockSize; br++)
        {
            for (int bc = 0; bc < BlockSize; bc++)
            {
                if (HasDuplicatesInBlock(grid.Cells, br, bc))
                {
                    throw new ArgumentException($"Grille invalide : doublons bloc ({br + 1}, {bc + 1})");
                }
            }
        }
    }

    private bool HasDuplicatesInRow(int[,] cells, int row)
    {
        var seen = new bool[GridSize + 1];
        for (int col = 0; col < GridSize; col++)
        {
            int val = cells[row, col];
            if (val != 0)
            {
                if (seen[val]) return true;
                seen[val] = true;
            }
        }
        return false;
    }

    private bool HasDuplicatesInColumn(int[,] cells, int col)
    {
        var seen = new bool[GridSize + 1];
        for (int row = 0; row < GridSize; row++)
        {
            int val = cells[row, col];
            if (val != 0)
            {
                if (seen[val]) return true;
                seen[val] = true;
            }
        }
        return false;
    }

    private bool HasDuplicatesInBlock(int[,] cells, int blockRow, int blockCol)
    {
        var seen = new bool[GridSize + 1];
        int startRow = blockRow * BlockSize;
        int startCol = blockCol * BlockSize;
        for (int i = 0; i < BlockSize; i++)
        {
            for (int j = 0; j < BlockSize; j++)
            {
                int val = cells[startRow + i, startCol + j];
                if (val != 0)
                {
                    if (seen[val]) return true;
                    seen[val] = true;
                }
            }
        }
        return false;
    }

    protected virtual IntVar[][] CreateCellVariables(Model model, SudokuGrid grid)
    {
        var variables = new IntVar[GridSize][];

        for (int row = 0; row < GridSize; row++)
        {
            variables[row] = new IntVar[GridSize];
            for (int col = 0; col < GridSize; col++)
            {
                int val = grid.Cells[row, col];
                if (val != 0)
                {
                    // Variable fixee a la valeur initiale
                    variables[row][col] = model.intVar($"cell_{row}_{col}", val);
                }
                else
                {
                    // Variable libre (1-9)
                    variables[row][col] = model.intVar($"cell_{row}_{col}", 1, GridSize, false);
                }
            }
        }
        return variables;
    }

    protected virtual void ApplyConstraints(Model model, IntVar[][] cellVariables)
    {
        // Contraintes sur les lignes et colonnes
        for (int i = 0; i < GridSize; i++)
        {
            model.allDifferent(cellVariables[i]).post();
            model.allDifferent(GetColumn(cellVariables, i)).post();
        }

        // Contraintes sur les blocs 3x3
        for (int blockRow = 0; blockRow < BlockSize; blockRow++)
        {
            for (int blockCol = 0; blockCol < BlockSize; blockCol++)
            {
                model.allDifferent(GetBlock(cellVariables, blockRow, blockCol)).post();
            }
        }
    }

    public virtual Solver GetSolver(Model model, IntVar[][] cellVariables)
    {
        // Strategie FirstFail + IntDomainMin
        model.getSolver().setSearch(
            org.chocosolver.solver.search.strategy.Search.intVarSearch(
                new FirstFail(model),
                new IntDomainMin(),
                FlatCells
            )
        );

        model.getSolver().setNoGoodRecordingFromRestarts();
        return model.getSolver();
    }

    protected virtual SudokuGrid ExtractSolution(SudokuGrid grid, IntVar[][] cells)
    {
        for (int row = 0; row < GridSize; row++)
        {
            for (int col = 0; col < GridSize; col++)
            {
                grid.Cells[row, col] = cells[row][col].getValue();
            }
        }
        return grid;
    }

    protected static IntVar[] Flatten(IntVar[][] matrix)
    {
        var flat = new IntVar[GridSize * GridSize];
        int index = 0;
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                flat[index++] = matrix[i][j];
            }
        }
        return flat;
    }

    private IntVar[] GetColumn(IntVar[][] grid, int col)
    {
        var column = new IntVar[GridSize];
        for (int row = 0; row < GridSize; row++)
        {
            column[row] = grid[row][col];
        }
        return column;
    }

    private IntVar[] GetBlock(IntVar[][] grid, int blockRow, int blockCol)
    {
        var block = new IntVar[BlockSize * BlockSize];
        int index = 0;
        int startRow = blockRow * BlockSize;
        int startCol = blockCol * BlockSize;
        for (int i = 0; i < BlockSize; i++)
        {
            for (int j = 0; j < BlockSize; j++)
            {
                block[index++] = grid[startRow + i][startCol + j];
            }
        }
        return block;
    }
}

Console.WriteLine("ChocoSolverVariableSelector defini avec strategie FirstFail.");

## 3. Test avec une Grille Exemple

In [ ]:
// Grille de test
var testPuzzle = new SudokuGrid();
testPuzzle.Cells = new int[,] {
    {9, 0, 2, 0, 0, 5, 4, 0, 3},
    {1, 0, 0, 0, 6, 3, 0, 2, 5},
    {5, 0, 8, 4, 0, 7, 0, 6, 0},
    {0, 2, 6, 3, 0, 9, 0, 0, 1},
    {0, 5, 7, 0, 1, 0, 2, 9, 0},
    {0, 9, 0, 6, 7, 0, 5, 3, 0},
    {2, 4, 0, 5, 3, 0, 6, 0, 0},
    {7, 0, 5, 2, 0, 0, 3, 0, 4},
    {0, 8, 0, 0, 4, 1, 9, 5, 0}
};

Console.WriteLine("Puzzle initial:");
Console.WriteLine(testPuzzle);

In [ ]:
// Test avec le solveur optimise
var solver = new ChocoSolverVariableSelector();
var stopwatch = System.Diagnostics.Stopwatch.StartNew();

var solution = solver.Solve(testPuzzle);
stopwatch.Stop();

Console.WriteLine($"\nSolution trouvee en {stopwatch.ElapsedMilliseconds} ms:");
Console.WriteLine(solution);

## 4. Comparaison des Strategies

### Heuristiques de Choix de Variables

| Strategie | Description | Complexite |
|-----------|-------------|------------|
| **InputOrder** | Ordre de declaration | O(1) |
| **FirstFail** | Domaine minimum (MRV) | O(n) |
| **DomOverWDeg** | Domaine / Degre pondere | O(n * d) |
| **ConflictHistory** | Historique des conflits | O(n * log n) |

### Heuristiques de Choix de Valeurs

| Strategie | Description |
|-----------|------------|
| **IntDomainMin** | Plus petite valeur |
| **IntDomainMax** | Plus grande valeur |
| **IntDomainRandom** | Valeur aleatoire |
| **IntDomainMiddle** | Valeur mediane |

## 5. Resume

### Points cles

1. **IKVM** permet d'utiliser Choco (Java) depuis C#
2. **Contrainte `allDifferent`** : 27 contraintes pour Sudoku (9 lignes + 9 colonnes + 9 blocs)
3. **Strategie FirstFail** : Choisir la variable avec le plus petit domaine (MRV)
4. **Performance** : Typiquement 1-50 ms selon la difficulte

### Comparaison avec autres solveurs

| Solveur | Langage | Performance | Facilite |
|---------|---------|-------------|----------|
| **Choco** | Java/C# (IKVM) | Moyenne | Moyenne |
| **OR-Tools** | C++/C# | Haute | Moyenne |
| **Z3** | C++/C# | Haute | Complexe |
| **python-constraint** | Python | Basse | Simple |

---

**Navigation** : [<< OR-Tools](Sudoku-10-ORTools-Csharp.ipynb) | [Index](README.md) | [Z3 >>](Sudoku-12-Z3-Csharp.ipynb)

*Code inspire du projet jsboigeECE/2025-ECE-Ing4-Fin-Sudoku-Gr01*